In [2]:
import ROOT as rt
import sys
import collections
from collections import OrderedDict
import uproot
import pandas as pd


import scipy
import awkward
import time

import subprocess
GIT_REPO = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
sys.path.append(GIT_REPO + '/lib/')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
from helper_functions import *

import numpy as np
from scipy.stats import norm
import math
import CMS_lumi, tdrstyle
style = tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0


# donotdelete = []
print(sys.version)

Welcome to JupyROOT 6.24/06
3.6.8 (default, Nov  2 2021, 13:01:57) 
[GCC 8.4.1 20200928 (Red Hat 8.4.1-1)]


In [3]:
%%time

fpath =OrderedDict()
tree = OrderedDict()

start_t = time.time()

# for data_year in ['2022','2023','all']:
for data_year in ['all']:

    # path = "/storage/af/user/christiw/login-1/christiw/LLP/Run3/CMSSW_14_0_1/src/mds_analysis/data/raw/"
    # fpath_bkg[data_year] = path + f"data_{data_year}_goodLumi.root"
    if data_year == 'all':path = f'/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/Run3/V1p19//Data_all/v14/normalized/'
    else: path = f'/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/Run3/V1p19//Data{data_year}/v14/normalized/'
    if data_year == '2022':fpath['data'+data_year] = path + "DisplacedJet-EXOCSCCluster_Run2022-PromptReco_goodLumi.root"
    elif data_year == '2023':fpath['data'+data_year] = path + "Muon-EXOCSCCluster_Run2023-PromptReco_goodLumi.root"
    elif data_year == 'all':fpath['data'+data_year] = path + "EXOCSCCluster_Run2022_2023_goodLumi.root"

fpath['sig'] = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/Run3/V1p19/MC_all/v14/normalized/ggH_Hto2Sto4B_MH-125-MS-15-ctauS-1000_TuneCP5_13p6TeV_powheg-pythia8_50000pb_weighted.root'
NEvents = {}

for k,v in fpath.items():
    print (k, v)
    root_dir = uproot.open(v) 

    tree[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'].values()[0]
    # NEvents[k] = root_dir['NEvents'].values()[0]
    # NEvents[k] = root_dir['NEvents'].counts()
    print("NEvents",NEvents[k])


dataall /storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/Run3/V1p19//Data_all/v14/normalized/EXOCSCCluster_Run2022_2023_goodLumi.root
NEvents 34804216.0
sig /storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/Run3/V1p19/MC_all/v14/normalized/ggH_Hto2Sto4B_MH-125-MS-15-ctauS-1000_TuneCP5_13p6TeV_powheg-pythia8_50000pb_weighted.root
NEvents 57555044.0
CPU times: user 351 ms, sys: 8.52 ms, total: 359 ms
Wall time: 358 ms


## CSC-CSC

In [44]:
%%time
MET_CUT = 200
name = [
"nCscRechitClusters = 2",
"HLTDecision",
"Offline HLT cut",
"Apply L1 efficiency weight",
f"MET < {MET_CUT}",
"jet veto",
"muon veto",
"ME1 veto",
"time spread",
"time",
"DNN",
"deltaPhi",
"CSC Hit",
    ]
for k,T in tree.items():
    print("************************************")
    print(k)
    print("************************************")
    # weight = T['weight'].array()* 52*27000/NEVENTS
    
    if 'data' in k: 
        weight = T['weight'].array()
        total = NEvents[k]
    else: 
        weight = T['pileupWeight'].array()*T['weight'].array()
        total = 52.2*50000

    presel =  (T["nCscRechitClusters"].array() == 2)
    print('\t'.join(str(x) for x in [name[0],  np.sum(weight[presel]), np.sum(weight[presel])/total, np.sum(weight[presel]/total)]))
    temp = np.sum(weight[presel])
    
    presel =  presel & T['HLT_CSCCSC'].array()
    print('\t'.join(str(x) for x in [name[1],  np.sum(weight[presel]), np.sum(weight[presel])/temp, np.sum(weight[presel]/total)]))
    temp = np.sum(weight[presel])
    
    cluster_pass_HLT = HLT_CSC(T['cscRechitClusterEta'].array(),T['cscRechitClusterNStation10'].array(),T['cscRechitClusterSize'].array())
    presel  = presel & (np.sum(cluster_pass_HLT, axis = 1) >= 1) # at least one passes HLT
    print('\t'.join(str(x) for x in [name[2],  np.sum(weight[presel]), np.sum(weight[presel])/temp, np.sum(weight[presel]/total)]))
    temp = np.sum(weight[presel])

    #apply L1 efficiency
    if 'sig' in k:
        HMTEff = T['cscRechitClusterHMTEfficiency'].array()
        HMTEff = 1-HMTEff[cluster_pass_HLT]
        HMTEff = 1-np.prod(HMTEff,axis=1)
        weight = weight * HMTEff
    print('\t'.join(str(x) for x in [name[3],  np.sum(weight[presel]), np.sum(weight[presel])/temp, np.sum(weight[presel]/total)]))
    temp = np.sum(weight[presel])

    presel = presel & (T['met'].array() < 200)
    print('\t'.join(str(x) for x in [name[4],  np.sum(weight[presel]), np.sum(weight[presel])/temp, np.sum(weight[presel]/total)]))
    temp = np.sum(weight[presel])

    me1 = (T['cscRechitClusterNRechitChamberPlus11'].array()+ T['cscRechitClusterNRechitChamberPlus12'].array()+\
        T['cscRechitClusterNRechitChamberMinus11'].array()+ T['cscRechitClusterNRechitChamberMinus12'].array())
    cluster = (T['cscRechitClusterJetVetoPt'].array()<30)
    presel = presel & (np.sum(cluster, axis = 1) ==2) # at least one passes HLT
    print('\t'.join(str(x) for x in [name[5],  np.sum(weight[presel]), np.sum(weight[presel])/temp, np.sum(weight[presel]/total)]))
    temp = np.sum(weight[presel])
    
    cluster = cluster & np.logical_not((T['cscRechitClusterMuonVetoPt'].array() > 30) & T['cscRechitClusterMuonVetoGlobal'].array())    
    presel = presel & (np.sum(cluster, axis = 1) ==2) # at least one passes HLT
    print('\t'.join(str(x) for x in [name[6],  np.sum(weight[presel]), np.sum(weight[presel])/temp, np.sum(weight[presel]/total)]))
    temp = np.sum(weight[presel])
    
    
    cluster = cluster & (me1 ==0)
    presel = presel & (np.sum(cluster, axis = 1) ==2) # at least one passes HLT
    print('\t'.join(str(x) for x in [name[7],  np.sum(weight[presel]), np.sum(weight[presel])/temp, np.sum(weight[presel]/total)]))
    temp = np.sum(weight[presel])
    
    cluster = cluster & (T['cscRechitClusterTimeSpreadWeightedAll'].array()<20)
    presel = presel & (np.sum(cluster, axis = 1) ==2) # at least one passes HLT
    print('\t'.join(str(x) for x in [name[8],  np.sum(weight[presel]), np.sum(weight[presel])/temp, np.sum(weight[presel]/total)]))
    temp = np.sum(weight[presel])
    
    cluster = cluster & (T['cscRechitClusterTimeWeighted'].array()< 12.5) & (T['cscRechitClusterTimeWeighted'].array()>-5)
    presel = presel & (np.sum(cluster, axis = 1) ==2) # at least one passes HLT
    print('\t'.join(str(x) for x in [name[9],  np.sum(weight[presel]), np.sum(weight[presel])/temp, np.sum(weight[presel]/total)]))
    temp = np.sum(weight[presel])
    
    clusterDNN = T['cscRechitClusterDNN_bkgMC_plusBeamHalo'].array()[cluster][presel][:,:2]
    sel = (np.sum((clusterDNN>0.96), axis = 1) ==2)
    print('\t'.join(str(x) for x in [name[10],  np.sum(weight[presel][sel]), np.sum(weight[presel][sel])/temp, np.sum(weight[presel][sel]/total)]))
    temp = np.sum(weight[presel][sel])
    
    cscRechitClusterPhi0 = T['cscRechitClusterPhi'].array()[cluster][presel][:,0]
    cscRechitClusterPhi1 = T['cscRechitClusterPhi'].array()[cluster][presel][:,1]
    deltaPhi_cluster = deltaPhi(np.array(cscRechitClusterPhi0), np.array(cscRechitClusterPhi1))    
    cscRechitClusterSize1 = T['cscRechitClusterSize'].array()[cluster][presel][:,1]
    sel = sel & (np.abs(deltaPhi_cluster)>2.8)
    print('\t'.join(str(x) for x in [name[11],  np.sum(weight[presel][sel]), np.sum(weight[presel][sel])/temp, np.sum(weight[presel][sel]/total)]))
    temp = np.sum(weight[presel][sel])
    
    if not 'data' in k: 
        sel = sel & (cscRechitClusterSize1 > 190)
        print('\t'.join(str(x) for x in [name[12],  np.sum(weight[presel][sel]), np.sum(weight[presel][sel])/temp, np.sum(weight[presel][sel]/total)]))
        temp = np.sum(weight[presel][sel])


************************************
dataall
************************************
nCscRechitClusters = 2	4605843.0	0.13233578	0.1323354
HLTDecision	4342808.0	0.942891	0.12477768
Offline HLT cut	3970750.0	0.9143278	0.114087775
Apply L1 efficiency weight	3970750.0	1.0	0.114087775
MET < 200	3969906.0	0.99978745	0.11406352
jet veto	3441473.0	0.8668903	0.09888066
muon veto	3440792.0	0.9998021	0.0988611
ME1 veto	183605.0	0.05336126	0.005275367
time spread	56695.0	0.30878788	0.0016289696
time	14806.0	0.2611518	0.00042540825
DNN	2199.0	0.14852087	6.3182e-05
deltaPhi	62.0	0.028194634	1.7813935e-06
************************************
sig
************************************
nCscRechitClusters = 2	45791.418	0.017544604585727968	0.017544605
HLTDecision	16375.878	0.35761893	0.006274283
Offline HLT cut	16357.229	0.9988612	0.006267137
Apply L1 efficiency weight	12811.836	0.78325224	0.0049087494
MET < 200	12633.35	0.98606867	0.0048403637
jet veto	10633.582	0.8417072	0.0040741693
muon veto	10626.713	0

## CSC-DT

In [9]:
%%time
CSC = 'cscRechitCluster'
DT = 'dtRechitCluster'

MET_CUT = 200
name = [
"nCscCluster = 1, nDtCluster = 1",
"HLTDecision",
"Offline HLT cut",
"Apply L1 efficiency weight",
f"MET < {MET_CUT}",
"jet veto",
"muon veto",
"ME1 veto",
"time spread",
"time",
"DNN",
"deltaPhi",
"CSC Hit",
    ]
for k,T in tree.items():
    print("************************************")
    print(k)
    print("************************************")
    # weight = T['weight'].array()* 52*27000/NEVENTS
    
    if 'data' in k: 
        weight = T['weight'].array()
        total = NEvents[k]
    else: 
        weight = T['pileupWeight'].array()*T['weight'].array()
        total = 52.2*50000

    presel =  (T["nCscRechitClusters"].array() == 1) & (T["nDtRechitClusters"].array() == 1)
    print('\t'.join(str(x) for x in [name[0],  np.sum(weight[presel]), np.sum(weight[presel])/total, np.sum(weight[presel]/total)]))
    temp = np.sum(weight[presel])
    
    presel =  presel & T['HLT_CSCDT'].array()
    print('\t'.join(str(x) for x in [name[1],  np.sum(weight[presel]), np.sum(weight[presel])/temp, np.sum(weight[presel]/total)]))
    temp = np.sum(weight[presel])
    

    #apply L1 efficiency
    if 'sig' in k:
        HMTEff = T['cscRechitClusterHMTEfficiency'].array()[presel][:,0]
        weight = weight[presel] * HMTEff
    else: weight = weight[presel]
    print('\t'.join(str(x) for x in [name[3],  np.sum(weight), np.sum(weight)/temp, np.sum(weight/total)]))
    temp = np.sum(weight)
    
    dPhi = deltaPhi(np.array(T['cscRechitClusterPhi'].array()[presel][:,0]), np.array(T['dtRechitClusterPhi'].array()[presel][:,0]))
    
    selections = {
        "met < 200": (T['met'].array() < 200)[presel],
        
        ### CSC cluster-level selections ###
        
        "CSC jet veto": (T[f'{CSC}JetVetoPt'].array()<30)[presel][:,0],
        "CSC muon veto": np.logical_not((T[f'{CSC}MuonVetoPt'].array() > 30) & T[f'{CSC}MuonVetoGlobal'].array())[presel][:,0],
        "CSC ME1 veto": ((T[f'{CSC}NRechitChamberPlus11'].array()+ T[f'{CSC}NRechitChamberPlus12'].array()+\
        T[f'{CSC}NRechitChamberMinus11'].array()+ T[f'{CSC}NRechitChamberMinus12'].array())==0)[presel][:,0],
        "CSC time spread": (T[f'{CSC}TimeSpreadWeightedAll'].array()<20)[presel][:,0],
        "CSC time cut": (T[f'{CSC}TimeWeighted'].array()< 12.5)[presel][:,0] & (T[f'{CSC}TimeWeighted'].array()>-5)[presel][:,0],
        "CSC DNN": (T[f'{CSC}DNN_bkgMC_plusBeamHalo'].array()>0.96)[presel][:,0],
        "CSC Nhits > 160": (T[f'{CSC}Size'].array()>160)[presel][:,0],
        
        ### DT cluster-level selections ###
        
        "DT jet veto":(T[f'{DT}JetVetoPt'].array()<30)[presel][:,0],
        "DT muon veto":np.logical_not((T[f'{DT}MuonVetoPt'].array() > 30) & T[f'{DT}MuonVetoLooseId'].array())[presel][:,0],
        "DT MB1 veto":(T[f'{DT}NHitStation1'].array()==0)[presel][:,0],
        "DT RPC hit":(T[f'{DT}_match_RPChits_dPhi0p5'].array() >= 1)[presel][:,0],
        "DT BX":(T[f'{DT}_match_RPCBx_dPhi0p5'].array() == 0)[presel][:,0],
        
        "deltaPhi": np.abs(dPhi) > 2.8,
        "DT Nhits > 90":(T[f'{DT}Size'].array() > 90)[presel][:,0],
    }
    
    for i, sel_k in enumerate(selections.keys()):
        if i == 0:sel = selections[sel_k]
        else: sel = sel & selections[sel_k]
        if i == len(selections.keys())-1 and 'data' in k:continue
        print('\t'.join(str(x) for x in [sel_k,  np.sum(weight[sel]), np.sum(weight[sel])/temp, np.sum(weight[sel]/total)]))
        temp = np.sum(weight[sel])
    
            
            

************************************
dataall
************************************
nCscCluster = 1, nDtCluster = 1	1549430.0	0.044518456	0.044518493
HLTDecision	750559.0	0.48440975	0.021565167
Apply L1 efficiency weight	750559.0	1.0	0.021565167
met < 200	750350.0	0.9997215	0.021559162
CSC jet veto	726516.0	0.96823615	0.020874362
CSC muon veto	726359.0	0.9997839	0.020869851
CSC ME1 veto	510851.0	0.70330375	0.014677855
CSC time spread	424325.0	0.8306238	0.012191773
CSC time cut	205732.0	0.48484534	0.0059111225
CSC DNN	54088.0	0.26290515	0.0015540648
CSC Nhits > 160	14038.0	0.25954002	0.00040334195
DT jet veto	13929.0	0.99223536	0.00040021015
DT muon veto	13929.0	1.0	0.00040021015
DT MB1 veto	5636.0	0.40462345	0.00016193441
DT RPC hit	2249.0	0.39904186	6.461861e-05
DT BX	619.0	0.27523345	1.7785202e-05
deltaPhi	19.0	0.03069467	5.4591084e-07
************************************
sig
************************************
nCscCluster = 1, nDtCluster = 1	74069.2	0.02837900502873563	0.028379012
HL